## Google Geocoding

* Now that everyone has an API Key (Congratulations!), it is time to start using it!

* You can utilize the Google Maps Geocoding API to turn addresses into latitudinal and longitudinal coordinates.

  * This process of converting an address to coordinates is called **geocoding**.

  * Since many APIs only understand locations formatted in terms of latitude/longitude, geocoding will be very valuable in translating addresses into data that APIs - like the Google Places API - can understand.

    * Google's API is not free and if credit card information is provided, Google will charge past a certain usage point. **It is a good idea to avoid pushing your API key to github by using adding the `config.py` to their `.gitignore` file or using environment variables.**

In [44]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey
from config import owm_api


file = pd.read_csv("winedata.csv")

file.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [2]:
file["winery name"] = file["winery"] + " Winery"
file.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,winery name
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Nicosia Winery
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos Winery
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Rainstorm Winery
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,St. Julian Winery
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Sweet Cheeks Winery


In [3]:
just_brazil = file.loc[file["country"]=="Brazil"]
just_brazil

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,winery name
5852,5852,Brazil,"Pear, apple and wet-stone aromas are tight, wh...",Brut,88,36.0,Pinto Bandeira,NaN,NaN,Michael Schachner,@wineschach,Cave Geisse 2014 Brut Sparkling (Pinto Bandeira),Sparkling Blend,Cave Geisse,Cave Geisse Winery
9151,9151,Brazil,This opens with dusty dried-apple aromas. High...,Brut,83,31.0,Santa Catarina,NaN,NaN,Michael Schachner,@wineschach,Villaggio Grando 2011 Brut Sparkling (Santa Ca...,Sparkling Blend,Villaggio Grando,Villaggio Grando Winery
10565,10565,Brazil,"The name is appropriate: this is intense, just...",Intenso,84,NaN,Campanha,NaN,NaN,Michael Schachner,@wineschach,Salton 2013 Intenso Tannat (Campanha),Tannat,Salton,Salton Winery
10579,10579,Brazil,Roasted gritty aromas of asphalt and grilled b...,Intenso,84,NaN,Campanha,NaN,NaN,Michael Schachner,@wineschach,Salton 2013 Intenso Cabernet Franc (Campanha),Cabernet Franc,Salton,Salton Winery
12669,12669,Brazil,This Cabernet Sauvignon-Merlot blend almost ha...,Reserva,83,20.0,Serra Gaúcha,NaN,NaN,Michael Schachner,@wineschach,Château Lacave 2002 Reserva Cabernet Sauvignon...,Cabernet Sauvignon-Merlot,Château Lacave,Château Lacave Winery
13218,13218,Brazil,"This leesy, bready, heavily toasted Brazilian ...",Brut 130,86,20.0,Vale dos Vinhedos,NaN,NaN,Michael Schachner,@wineschach,Casa Valduga NV Brut 130 (Vale dos Vinhedos),Champagne Blend,Casa Valduga,Casa Valduga Winery
14726,14726,Brazil,Bell pepper and sharp red fruit aromas provide...,Proprietary Red,82,10.0,Serra Gaúcha,NaN,NaN,Michael Schachner,@wineschach,Aurora Cellars 2004 Proprietary Red Red (Serra...,Bordeaux-style Red Blend,Aurora Cellars,Aurora Cellars Winery
14734,14734,Brazil,"A weird Cabernet with unfamiliar, odd aromas. ...",Gran Reserva,81,36.0,Vale dos Vinhedos,NaN,NaN,Michael Schachner,@wineschach,Casa Valduga 2007 Gran Reserva Cabernet Sauvig...,Cabernet Sauvignon,Casa Valduga,Casa Valduga Winery
14739,14739,Brazil,"Sketchy right away, with salinic, pool water a...",Da'divas,80,20.0,Vale dos Vinhedos,NaN,NaN,Michael Schachner,@wineschach,Lidio Carraro 2010 Da'divas Chardonnay (Vale d...,Chardonnay,Lidio Carraro,Lidio Carraro Winery
15989,15989,Brazil,Aromas of lemon-lime and lemon-scented househo...,Cave Amadeu,86,25.0,Pinto Bandeira,NaN,NaN,Michael Schachner,@wineschach,Cave Geisse NV Cave Amadeu Moscatel (Pinto Ban...,Moscatel,Cave Geisse,Cave Geisse Winery


In [4]:
# Target location

winery_response =[]


for winery in just_brazil["winery name"]:

    target_location = winery
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    
    print(geo_data)
   
    winery_response.append(geo_data)

        
   
        

{'results': [{'address_components': [{'long_name': 'Pinto Bandeira', 'short_name': 'Pinto Bandeira', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95717-000', 'short_name': '95717-000', 'types': ['postal_code']}], 'formatted_address': 'Linha Jansen, s/n - Zona Rural, Pinto Bandeira - RS, 95717-000, Brazil', 'geometry': {'location': {'lat': -29.139567, 'lng': -51.42783799999999}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': {'lat': -29.1382180197085, 'lng': -51.4264890197085}, 'southwest': {'lat': -29.1409159802915, 'lng': -51.4291869802915}}}, 'place_id': 'ChIJzZfiqFkhHJURY356ZbvCz5A', 'plus_code': {'compound_code': 'VH6C+5V Bento Gonçalves, State of Rio Grande do Sul, Brazil', 'global_code': '582CVH6C+5V'}, 'types': ['establishment', 'food', 'point_of_int

{'results': [{'address_components': [{'long_name': '2355', 'short_name': '2355', 'types': ['street_number']}, {'long_name': 'Via Trento', 'short_name': 'Via Trento', 'types': ['route']}, {'long_name': 'Bento Gonçalves', 'short_name': 'Bento Gonçalves', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95700-000', 'short_name': '95700-000', 'types': ['postal_code']}], 'formatted_address': 'Via Trento, 2355, Bento Gonçalves - RS, 95700-000, Brazil', 'geometry': {'location': {'lat': -29.176995, 'lng': -51.556275}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -29.1756460197085, 'lng': -51.5549260197085}, 'southwest': {'lat': -29.1783439802915, 'lng': -51.5576239802915}}}, 'place_id': 'ChIJM0Iv4X08HJURcdhd86DQSsQ', 'plus_code': {'compound_code': 'RCFV+6F Bento Gonça

{'results': [{'address_components': [{'long_name': 's/n', 'short_name': 's/n', 'types': ['street_number']}, {'long_name': 'Localidade Santos Anjos', 'short_name': 'Localidade Santos Anjos', 'types': ['route']}, {'long_name': 'Quarto Distrito', 'short_name': 'Quarto Distrito', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Farroupilha', 'short_name': 'Farroupilha', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95170-971', 'short_name': '95170-971', 'types': ['postal_code']}], 'formatted_address': 'Localidade Santos Anjos, s/n - Quarto Distrito, Farroupilha - RS, 95170-971, Brazil', 'geometry': {'location': {'lat': -29.2555705, 'lng': -51.2917757}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -29.2542215197085, 'lng': -51.290426

{'results': [{'address_components': [{'long_name': 'KM 21', 'short_name': 'KM 21', 'types': ['street_number']}, {'long_name': 'RS-444', 'short_name': 'RS-444', 'types': ['route']}, {'long_name': 'Vale dos Vinhedos', 'short_name': 'Vale dos Vinhedos', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Bento Gonçalves', 'short_name': 'Bento Gonçalves', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95700-000', 'short_name': '95700-000', 'types': ['postal_code']}], 'formatted_address': 'RS-444, KM 21 - Vale dos Vinhedos, Bento Gonçalves - RS, 95700-000, Brazil', 'geometry': {'location': {'lat': -29.1819131, 'lng': -51.5843564}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -29.1805641197085, 'lng': -51.5830074197085}, 'southwest': {'la

{'results': [{'address_components': [{'long_name': 's/n', 'short_name': 's/n', 'types': ['street_number']}, {'long_name': 'Localidade Santos Anjos', 'short_name': 'Localidade Santos Anjos', 'types': ['route']}, {'long_name': 'Quarto Distrito', 'short_name': 'Quarto Distrito', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Farroupilha', 'short_name': 'Farroupilha', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95170-971', 'short_name': '95170-971', 'types': ['postal_code']}], 'formatted_address': 'Localidade Santos Anjos, s/n - Quarto Distrito, Farroupilha - RS, 95170-971, Brazil', 'geometry': {'location': {'lat': -29.2555705, 'lng': -51.2917757}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -29.2542215197085, 'lng': -51.290426

{'results': [{'address_components': [{'long_name': '2355', 'short_name': '2355', 'types': ['street_number']}, {'long_name': 'Via Trento', 'short_name': 'Via Trento', 'types': ['route']}, {'long_name': 'Bento Gonçalves', 'short_name': 'Bento Gonçalves', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Rio Grande do Sul', 'short_name': 'RS', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '95700-000', 'short_name': '95700-000', 'types': ['postal_code']}], 'formatted_address': 'Via Trento, 2355, Bento Gonçalves - RS, 95700-000, Brazil', 'geometry': {'location': {'lat': -29.176995, 'lng': -51.556275}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -29.1756460197085, 'lng': -51.5549260197085}, 'southwest': {'lat': -29.1783439802915, 'lng': -51.5576239802915}}}, 'place_id': 'ChIJM0Iv4X08HJURcdhd86DQSsQ', 'plus_code': {'compound_code': 'RCFV+6F Bento Gonça

{'results': [{'address_components': [{'long_name': '5973', 'short_name': '5973', 'types': ['street_number']}, {'long_name': 'Engineer Drive', 'short_name': 'Engineer Dr', 'types': ['route']}, {'long_name': 'Huntington Beach', 'short_name': 'Huntington Beach', 'types': ['locality', 'political']}, {'long_name': 'Orange County', 'short_name': 'Orange County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '92649', 'short_name': '92649', 'types': ['postal_code']}, {'long_name': '1129', 'short_name': '1129', 'types': ['postal_code_suffix']}], 'formatted_address': '5973 Engineer Dr, Huntington Beach, CA 92649, USA', 'geometry': {'location': {'lat': 33.7429446, 'lng': -118.0250535}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 33.7442935802915, 'lng': -118.023704519

{'results': [{'address_components': [{'long_name': 'S/N', 'short_name': 'S/N', 'types': ['street_number']}, {'long_name': 'Rodovia SC-350, KM 109', 'short_name': 'Rodovia SC-350, KM 109', 'types': ['route']}, {'long_name': 'Água Doce', 'short_name': 'Água Doce', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Santa Catarina', 'short_name': 'SC', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}, {'long_name': '89654-000', 'short_name': '89654-000', 'types': ['postal_code']}], 'formatted_address': 'Rodovia SC-350, KM 109, S/N, Água Doce - SC, 89654-000, Brazil', 'geometry': {'location': {'lat': -26.7256295, 'lng': -51.50959899999999}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': -26.7242805197085, 'lng': -51.5082500197085}, 'southwest': {'lat': -26.7269784802915, 'lng': -51.51094798029149}}}, 'place_id': 'ChIJ_yAhbg0Y5JQRyfAsoS9O3Bg', 'plus_code': {'compound_code'

In [5]:
# Pretty print the json
pprint(winery_response)

[{'results': [{'address_components': [{'long_name': 'Pinto Bandeira',
                                       'short_name': 'Pinto Bandeira',
                                       'types': ['administrative_area_level_2',
                                                 'political']},
                                      {'long_name': 'Rio Grande do Sul',
                                       'short_name': 'RS',
                                       'types': ['administrative_area_level_1',
                                                 'political']},
                                      {'long_name': 'Brazil',
                                       'short_name': 'BR',
                                       'types': ['country', 'political']},
                                      {'long_name': '95717-000',
                                       'short_name': '95717-000',
                                       'types': ['postal_code']}],
               'formatted_address': 'Linha Ja

                                      {'long_name': 'Riverside County',
                                       'short_name': 'Riverside County',
                                       'types': ['administrative_area_level_2',
                                                 'political']},
                                      {'long_name': 'California',
                                       'short_name': 'CA',
                                       'types': ['administrative_area_level_1',
                                                 'political']},
                                      {'long_name': 'United States',
                                       'short_name': 'US',
                                       'types': ['country', 'political']},
                                      {'long_name': '92254',
                                       'short_name': '92254',
                                       'types': ['postal_code']},
                                      {'long_name'

                                      {'long_name': 'California',
                                       'short_name': 'CA',
                                       'types': ['administrative_area_level_1',
                                                 'political']},
                                      {'long_name': 'United States',
                                       'short_name': 'US',
                                       'types': ['country', 'political']},
                                      {'long_name': '92274',
                                       'short_name': '92274',
                                       'types': ['postal_code']},
                                      {'long_name': '9654',
                                       'short_name': '9654',
                                       'types': ['postal_code_suffix']}],
               'formatted_address': '91711 82nd Ave, Thermal, CA 92274, USA',
               'geometry': {'location': {'lat': 33.453381, 'lng': 

                                              'Brazil',
                             'global_code': '585C7FFR+P5'},
               'types': ['establishment', 'food', 'point_of_interest']}],
  'status': 'OK'},
 {'results': [{'address_components': [{'long_name': 's/n',
                                       'short_name': 's/n',
                                       'types': ['street_number']},
                                      {'long_name': 'Localidade Santos Anjos',
                                       'short_name': 'Localidade Santos Anjos',
                                       'types': ['route']},
                                      {'long_name': 'Quarto Distrito',
                                       'short_name': 'Quarto Distrito',
                                       'types': ['political',
                                                 'sublocality',
                                                 'sublocality_level_1']},
                                      {'lo

In [45]:
# Collect and store latitude and longitude for each winery
lat = []
lng = []

i=0

# Loop through wineries and append lists
for w in winery_response:
    try:
        latitude = winery_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = winery_response[i]["results"][0]["geometry"]["location"]["lng"]
        
        
        lat.append(latitude)
        lng.append(longitude)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
    
        i = i+1    